# Bootcamp Data Science Aplicada - Alura

# Módulo 02 - Visualização de Dados com Seaborn e Matplotlib

**Obs.** Notabook para o acompanhamento das aulas e resolução dos desafios propostos. Caso necessário, dividirei em mais de um notebook.

## Aula 01 - Fontes Externas, Limpeza e Manipulação de Dados

Iniciou-se revisando o notebook do módulo anterior e comparando o valor "bruto" com o valor absoluto de gastos.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from io import StringIO

# bibliotecas utilizadas na parte de web scraping
import requests
from bs4 import BeautifulSoup

# fomatação dos números para duas casas decimais
pd.options.display.float_format = "{:.2f}".format

### Resumo dos dataframes usados no módulo passado, para caso algum seja reutilizado nesse módulo 2

In [2]:
dados = pd.read_csv("https://raw.githubusercontent.com/alura-cursos/agendamento-hospitalar/main/dados/A160324189_28_143_208.csv",
                    encoding = 'ISO-8859-1', skiprows = 3, sep = ';',
                    skipfooter = 12, thousands = '.', decimal = ',', engine = 'python')

In [3]:
colunas_usaveis = dados.mean().index.tolist()
colunas_usaveis.insert(0, "Unidade da Federação")
usaveis = dados[colunas_usaveis]
usaveis = usaveis.set_index("Unidade da Federação")
usaveis = usaveis.drop("Total", axis = 1)
usaveis["Total"] = usaveis.sum(axis = 1)
ordenado_por_total = usaveis.sort_values("Total", ascending = False)
ordenado_por_total = ordenado_por_total.drop("Total", axis = 1)
ordenado_por_total = ordenado_por_total / 1000000
meses_filtrados = ordenado_por_total.columns[47:150]
mes_mais_recente = ordenado_por_total.columns[-1]
gastos_do_mais_recente = ordenado_por_total[mes_mais_recente]
gastos_do_mais_recente = gastos_do_mais_recente.sort_values(ascending = False)
gastos_proporcionais = gastos_do_mais_recente / gastos_do_mais_recente.sum()

### Aulas 01 & 02

In [4]:
ibge_estimativa = pd.read_excel("https://github.com/alura-cursos/agendamento-hospitalar/raw/main/dados/estimativa_dou_2020.xls")
ibge_estimativa.head()

,ESTIMATIVAS DA POPULAÇÃO RESIDENTE NO BRASIL E UNIDADES DA FEDERAÇÃO COM DATA DE REFERÊNCIA EM 1º DE JULHO DE 2020,Unnamed: 1,Unnamed: 2
0,BRASIL E UNIDADES DA FEDERAÇÃO,NaN,POPULAÇÃO ESTIMADA
1,Brasil,NaN,211755692
2,Região Norte,NaN,18672591
3,Rondônia,NaN,1796460
4,Acre,NaN,894470


In [5]:
dados_da_populacao = """Posição	Unidade federativa	População	% da pop. total	País comparável
(habitantes)

1	 São Paulo	46 289 333	21,9%	Flag of Spain.svg Espanha (46 439 864)
2	 Minas Gerais	21 292 666	10,1%	 Sri Lanka (20 675 000)
3	 Rio de Janeiro	17 366 189	8,2%	 Países Baixos (16 922 900)
4	Bahia Bahia	14 930 634	7,1%	 Chade (14 037 000)
5	 Paraná	11 516 840	5,4%	 Bolívia (11 410 651)
6	 Rio Grande do Sul	11 422 973	5,4%	 Bélgica (11 250 659)
7	 Pernambuco	9 616 621	4,5%	 Bielorrússia (9 485 300)
8	 Ceará	9 187 103	4,3%	 Emirados Árabes Unidos (9 157 000)
9	Pará Pará	8 690 745	4,1%	 Áustria (8 602 112)
10	 Santa Catarina	7 252 502	3,4%	 Sérvia (7 114 393)
11	 Maranhão	7 114 598	3,4%	 Paraguai (7 003 406)
12	 Goiás	7 113 540	3,4%	 Paraguai (7 003 406)
13	 Amazonas	4 207 714	2,0%	 Líbano (4 168 000)
14	 Espírito Santo	4 064 052	1,9%	 Líbano (4 168 000)
15	 Paraíba	4 039 277	1,9%	 Líbano (4 168 000)
16	 Rio Grande do Norte	3 534 165	1,7%	 Uruguai (3 415 866)
17	 Mato Grosso	3 526 220	1,7%	 Uruguai (3 415 866)
18	 Alagoas	3 351 543	1,6%	 Uruguai (3 415 866)
19	 Piauí	3 281 480	1,6%	 Kuwait (3 268 431)
20	 Distrito Federal	3 055 149	1,4%	 Lituânia (2 900 787)
21	 Mato Grosso do Sul	2 809 394	1,3%	 Jamaica (2 717 991)
22	 Sergipe	2 318 822	1,1%	 Namíbia (2 280 700)
23	 Rondônia	1 796 460	0,8%	 Gabão (1 725 000)
24	 Tocantins	1 590 248	0,7%	 Bahrein (1 359 800)
25	 Acre	894 470	0,4%	 Fiji (859 178)
26	 Amapá	861 773	0,4%	 Fiji (859 178)
27	 Roraima	631 181	0,3%	 Luxemburgo (562 958)"""

# fonte: https://pt.wikipedia.org/wiki/Lista_de_unidades_federativas_do_Brasil_por_popula%C3%A7%C3%A3o

- Um referência interessante sobre **StringIO**: [Manipulando Strings Como Se Fossem Arquivos](https://pythonhelp.wordpress.com/2014/06/19/manipulando-strings-como-se-fossem-arquivos-stringio/)

In [6]:
# converte a string para stringIO
dados_da_populacao_io = StringIO(dados_da_populacao)

# lê os dados que colamos acima em um pandas dataframe
populacao = pd.read_csv(dados_da_populacao_io, sep="\t")

# dropar os valores NaN
populacao = populacao.dropna()

# remove os espaços nos números
populacao["populacao"] = populacao["População"].str.replace(" ", "").astype(int)

# muda o nome da coluna
populacao["uf"] = populacao["Unidade federativa"]

# coloca a uf como índice
populacao = populacao[["populacao", "uf"]].set_index("uf")


display(populacao.head())
display(populacao.info())

,populacao
uf,
São Paulo,46289333
Minas Gerais,21292666
Rio de Janeiro,17366189
Bahia Bahia,14930634
Paraná,11516840


<class 'pandas.core.frame.DataFrame'>
Index: 27 entries,  São Paulo to  Roraima
Data columns (total 1 columns):
 #   Column     Non-Null Count  Dtype
---  ------     --------------  -----
 0   populacao  27 non-null     int64
dtypes: int64(1)
memory usage: 432.0+ bytes


None

In [7]:
gastos_do_mais_recente.index = gastos_do_mais_recente.index.str[3:]

In [8]:
populacao.index = populacao.index.str.strip()

In [9]:
for estado in gastos_do_mais_recente.index:
  # print(f"{estado} {estado} => {estado}")
  populacao.index = populacao.index.str.replace(f"{estado} {estado}", estado)
populacao.index

Index(['São Paulo', 'Minas Gerais', 'Rio de Janeiro', 'Bahia', 'Paraná',
       'Rio Grande do Sul', 'Pernambuco', 'Ceará', 'Pará', 'Santa Catarina',
       'Maranhão', 'Goiás', 'Amazonas', 'Espírito Santo', 'Paraíba',
       'Rio Grande do Norte', 'Mato Grosso', 'Alagoas', 'Piauí',
       'Distrito Federal', 'Mato Grosso do Sul', 'Sergipe', 'Rondônia',
       'Tocantins', 'Acre', 'Amapá', 'Roraima'],
      dtype='object', name='uf')

In [10]:
gastos_e_populacao_recente = populacao.join(gastos_do_mais_recente)
gastos_e_populacao_recente.head()

,populacao,2020/Jul
uf,,
São Paulo,46289333,282.71
Minas Gerais,21292666,120.47
Rio de Janeiro,17366189,71.54
Bahia,14930634,58.90
Paraná,11516840,80.32


### Desafio 01 - Entre nos dados da Wikipédia e leia a base de dados direto da Wikipédia.

Para resolver esse desafio precisamos usar da técnica de **_web scraping_**.

- Link para obtenção dos dados: https://pt.wikipedia.org/wiki/Lista_de_unidades_federativas_do_Brasil_por_popula%C3%A7%C3%A3o

- [Web Scraping a Wikipedia Table into a Dataframe](https://medium.com/analytics-vidhya/web-scraping-a-wikipedia-table-into-a-dataframe-c52617e1f451)

In [11]:
# url que utilizaremos
url_wiki = 'https://pt.wikipedia.org/wiki/Lista_de_unidades_federativas_do_Brasil_por_popula%C3%A7%C3%A3o'

# fazemos uma 'requisição' na página e guardamos em 'response'
response = requests.get(url_wiki)

Há muitas controvérsias sobre a legalidade de se realizar web scraping, mas, no nosso caso, a Wikipédia permite realizarmos o _web scraping_ para fins de estudo, e de uma forma responsável, sem mandar muitos _requests_ ao mesmo tempo para a página.

Então vamos em frente.

<img width = "1400" src="https://raw.githubusercontent.com/diascarolina/data-science-bootcamp/main/modulo2/inspect-element.png" />

Na imagem vemos o que obtemos ao clicarmos com o botão direito do mouse na página da Wikipédia que queremos obter os dados e escolhermos "Inspecionar Elemento". Isso mostra as tags HTML de cada objeto. Observe que, na tabela em si, temos a tag ```<table>```. Essa informação é crucial na hora de realizarmos o _web scraping_.

In [12]:
# instância do BeautifulSoup
soup = BeautifulSoup(response.text, 'html.parser')

# pega a tabela
populacao_tabela = soup.find('table', {'class':"wikitable"})

In [13]:
# lê a tabela com o pandas
pop = pd.read_html(str(populacao_tabela))

# transforma num pandas dataframe
pop = pd.DataFrame(pop[0])

In [14]:
pop.head()

,Posição,Unidade federativa,População,% da pop. total,País comparável (habitantes)
0,1,São Paulo,46 289 333,"21,9%",Espanha (46 439 864)
1,2,Minas Gerais,21 292 666,"10,1%",Sri Lanka (20 675 000)
2,3,Rio de Janeiro,17 366 189,"8,2%",Países Baixos (16 922 900)
3,4,Bahia,14 930 634,"7,1%",Chade (14 037 000)
4,5,Paraná,11 516 840,"5,4%",Bolívia (11 410 651)


Agora temos um pandas dataframe com as informações diretamente da Wikipédia. Vamos limpá-lo um pouco e salvá-lo em um arquivo .csv.

In [15]:
pop.columns

Index(['Posição', 'Unidade federativa', 'População', '% da pop. total',
       'País comparável (habitantes)'],
      dtype='object')

In [16]:
pop = pop.drop(['Posição', '% da pop. total', 'País comparável (habitantes)'], axis = 1)

In [17]:
pop.columns = ['Unidade da Federacao', 'habitantes']

# nessa parte tive diversos bugs pois inicialmente fiz uma listas de listas, quando é necessário apenas uma única lista

In [18]:
pop["habitantes"] = pop["habitantes"].str.replace(" ", "").astype(int)

In [19]:
pop.head()

,Unidade da Federacao,habitantes
0,São Paulo,46289333
1,Minas Gerais,21292666
2,Rio de Janeiro,17366189
3,Bahia,14930634
4,Paraná,11516840


In [20]:
pop = pop.set_index('Unidade da Federacao').rename_axis(None)

In [22]:
pop.head()

,habitantes
São Paulo,46289333
Minas Gerais,21292666
Rio de Janeiro,17366189
Bahia,14930634
Paraná,11516840


Ufa! Finalmente temos nosso dataframe limpo. Vamos salvá-lo em formato .csv.

In [33]:
# descomentar para salvar o csv
# pop.to_csv(r'../data/populacao_estados_br.csv')

### Desafio 02 - Neste desafio temos uma série de passos:
- Padronizar os nomes das colunas
- Transformar os dados nos tipos corretos
- Transformar a UF em index

Queremos fazer isso com os dois dataframes que temos: ```pop``` e ```gastos_do_mais_recente```.

In [26]:
pop.head()

,habitantes
São Paulo,46289333
Minas Gerais,21292666
Rio de Janeiro,17366189
Bahia,14930634
Paraná,11516840


In [32]:
pop.dtypes

habitantes    int64
dtype: object

In [28]:
gastos_do_mais_recente.head()

Unidade da Federação
São Paulo           282.71
Minas Gerais        120.47
Paraná               80.32
Rio Grande do Sul    78.93
Rio de Janeiro       71.54
Name: 2020/Jul, dtype: float64

Os dois dataframes já estão pronto para o ```join```, pois realizamos a limpeza anteriormente.

In [29]:
gastos_e_pop_recente = pop.join(gastos_do_mais_recente)

In [30]:
gastos_e_pop_recente.head()

,habitantes,2020/Jul
São Paulo,46289333,282.71
Minas Gerais,21292666,120.47
Rio de Janeiro,17366189,71.54
Bahia,14930634,58.90
Paraná,11516840,80.32


### Desafio 03 - Buscar na documentação alguns métodos envolvendo strings

- [Python String Methods](https://www.w3schools.com/python/python_ref_string.asp)

- [Pandas Documentation on .str](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.str.html)

Entre os métodos que achei mais interessantes estão:
- ```count()``` -> Returns the number of times a specified value occurs in a string
- ```encode()``` -> Returns an encoded version of the string
- ```isnumeric()``` -> Returns True if all characters in the string are numeric

### Desafio 04 - Desenvolver uma função para a limpeza e tratamento dos dados, retornando duas tabelas "população" e "unida".

Acho que este desafio não está bem definido, pois está vago. Entendi que devemos fazer uma função especificamente para os dados utilizados nessa aula.

In [43]:
def data_cleaning(url_dados, url_wiki):
    '''
    Recebe duas urls. A primeira é a url dos dados em .csv
    e a segunda é a url da pagina da Wikipédia para scraping
    Retorna dois dataframes:
        - populacao: dataframe obtido atráves da url
        - unida: um dataframe com os dados populacionais e os dados fornecidos originalmente
    '''
    
    # leitura e limpeza da url_dados
    dados = pd.read_csv(url_dados, encoding = 'ISO-8859-1', skiprows = 3, sep = ';',
                    skipfooter = 12, thousands = '.', decimal = ',', engine = 'python')
    
    colunas_usaveis = dados.mean().index.tolist()
    colunas_usaveis.insert(0, "Unidade da Federação")
    usaveis = dados[colunas_usaveis]
    usaveis = usaveis.set_index("Unidade da Federação")
    usaveis = usaveis.drop("Total", axis = 1)
    usaveis["Total"] = usaveis.sum(axis = 1)
    ordenado_por_total = usaveis.sort_values("Total", ascending = False)
    ordenado_por_total = ordenado_por_total.drop("Total", axis = 1)
    ordenado_por_total = ordenado_por_total / 1000000
    meses_filtrados = ordenado_por_total.columns[47:150]
    mes_mais_recente = ordenado_por_total.columns[-1]
    gastos_do_mais_recente = ordenado_por_total[mes_mais_recente]
    gastos_do_mais_recente = gastos_do_mais_recente.sort_values(ascending = False)
    gastos_do_mais_recente.index = gastos_do_mais_recente.index.str[3:]
    
    # web scraping part
    response = requests.get(url_wiki)
    soup = BeautifulSoup(response.text, 'html.parser')
    populacao_tabela = soup.find('table', {'class':"wikitable"})
    populacao = pd.read_html(str(populacao_tabela))
    populacao = pd.DataFrame(populacao[0])
    
    # limpeza dessa tabela
    populacao = populacao[['Unidade federativa', 'População']]
    populacao.columns = ['UF', 'pop']
    populacao["pop"] = populacao["pop"].str.replace(" ", "").astype(int)
    populacao = populacao.set_index('UF').rename_axis(None)
    
    
    # o dataframe 'gastos_do_mais_recente' já está limpo
    # então já podemos fazer um join com 'populacao'
    unida = populacao.join(gastos_do_mais_recente)
    
    return populacao, unida   

In [44]:
url_dados = 'https://raw.githubusercontent.com/alura-cursos/agendamento-hospitalar/main/dados/A160324189_28_143_208.csv'
url_wikipedia = 'https://pt.wikipedia.org/wiki/Lista_de_unidades_federativas_do_Brasil_por_popula%C3%A7%C3%A3o'

populacao_teste, unida_teste = data_cleaning(url_dados, url_wikipedia)

In [45]:
populacao_teste.head()

,pop
São Paulo,46289333
Minas Gerais,21292666
Rio de Janeiro,17366189
Bahia,14930634
Paraná,11516840


In [46]:
unida_teste.head()

,pop,2020/Jul
São Paulo,46289333,282.71
Minas Gerais,21292666,120.47
Rio de Janeiro,17366189,71.54
Bahia,14930634,58.90
Paraná,11516840,80.32


### O que aprendemos nessa aula:
- Como comparar os gastos em relação a determinado estado;
- Como fazer a leitura de dados no formato Excel com Pandas;
- Como tornar uma string em (com formato de tabela) legível em pandas usando o String.IO;
- Como usar Join em Pandas.

## Aula 02 - Visualização, Seaborn e Proporcionalidades